In [2]:
'''basic packages'''
import os
import pandas as pd
import numpy as np
from sklearn import datasets, tree, linear_model
import sklearn.ensemble as ek
import seaborn as sns
import queue
import cv2          #影像處理
import scipy.misc   #影像處理
from tqdm import tqdm_notebook as tqdm #進度條
import matplotlib.pyplot as plt
plt.rcParams['font.family']='SimHei' #顯示中文
import matplotlib.lines as mlines
%matplotlib inline
import warnings
# warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning)

from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [3]:
# Load in the train datasets
train = pd.read_csv('/data/examples/may_the_4_be_with_u/show_me_the_money/train.csv',
                    names=['id','step','type','amount','nameOrig','oldbalanceOrg','newbalanceOrig','nameDest','oldbalanceDest','newbalanceDest','Fraud'],
                    encoding = "utf-8", header=None)
test = pd.read_csv('/data/examples/may_the_4_be_with_u/show_me_the_money/test.csv',
                   names=['id','step','type','amount','nameOrig','oldbalanceOrg','newbalanceOrig','nameDest','oldbalanceDest','newbalanceDest'],
                   encoding = "utf-8", header=None)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,1,3,5,6,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,1,3,5,6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train.shape

(3817573, 11)

In [5]:
train.isnull().values.any()

False

In [6]:
train.head()

,id,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,Fraud
0,id,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,Fraud
1,0,323,PAYMENT,9092.69,C749189758,0.0,0.0,M1939669337,0.0,0.0,0.0
2,1,164,CASH_IN,23468.75,C525009948,0.0,23468.75,C65353523,77300.17,53831.42,0.0
3,2,228,CASH_OUT,39688.11,C1311063915,0.0,0.0,C1581032686,168913.26,208601.37,0.0
4,3,229,CASH_OUT,179016.96,C700594269,103485.0,0.0,C90149178,2483034.47,2662051.42,0.0


In [7]:
train = train.drop([0])

In [8]:
train.head()

,id,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,Fraud
1,0,323,PAYMENT,9092.69,C749189758,0.0,0.0,M1939669337,0.0,0.0,0.0
2,1,164,CASH_IN,23468.75,C525009948,0.0,23468.75,C65353523,77300.17,53831.42,0.0
3,2,228,CASH_OUT,39688.11,C1311063915,0.0,0.0,C1581032686,168913.26,208601.37,0.0
4,3,229,CASH_OUT,179016.96,C700594269,103485.0,0.0,C90149178,2483034.47,2662051.42,0.0
5,4,709,CASH_OUT,77155.99,C646560157,143307.0,66151.01,C671337546,21213865.83,21291021.82,0.0


In [9]:
test.shape

(2545048, 10)

In [10]:
test.isnull().values.any()

False

In [11]:
test.head()

,id,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest
0,id,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest
1,1,329,PAYMENT,41909.68,C1965438439,18391.0,0.0,M296912999,0.0,0.0
2,2,205,CASH_IN,57645.11,C2061644565,3162376.19,3220021.31,C2046839454,701205.57,643617.16
3,3,299,PAYMENT,41911.35,C630506216,9218.0,0.0,M1895361660,0.0,0.0
4,4,165,PAYMENT,10297.71,C1059151844,29739.0,19441.29,M830221248,0.0,0.0


### 匯出
train.to_csv('show_me_the_money_train.csv',index=False)

test.to_csv('show_me_the_money_test.csv',index=False)

In [12]:
print('\n The types of fraudulent transactions are {}'.format(\
list(train.loc[train.Fraud == 1].type.drop_duplicates().values))) # only 'CASH_OUT' 
                                                             # & 'TRANSFER'

dfFraudTransfer = train.loc[(train.Fraud == 1) & (train.type == 'TRANSFER')]
dfFraudCashout = train.loc[(train.Fraud == 1) & (train.type == 'CASH_OUT')]

print ('\n The number of fraudulent TRANSFERs = {}'.\
       format(len(dfFraudTransfer))) # 4097

print ('\n The number of fraudulent CASH_OUTs = {}'.\
       format(len(dfFraudCashout))) # 4116


 The types of fraudulent transactions are ['CASH_OUT', 'TRANSFER']

 The number of fraudulent TRANSFERs = 2421

 The number of fraudulent CASH_OUTs = 2426


In [13]:
print('\nThe type of transactions in which Fraud is set: \
{}'.format(list(train.loc[train.Fraud == 1].type.drop_duplicates()))) 
                                                            # only 'TRANSFER'

dfTransfer = train.loc[train.type == 'TRANSFER']
dfFlagged = train.loc[train.Fraud == 1]
dfNotFlagged = train.loc[train.Fraud == 0]

print('\nMin amount transacted when Fraud is set= {}'\
                                  .format(dfFlagged.amount.min())) # 353874.22

print('\nMax amount transacted in a TRANSFER where Fraud is not set=\
 {}'.format(dfTransfer.loc[dfTransfer.Fraud == 0].amount.max())) # 92445516.64


The type of transactions in which Fraud is set: ['CASH_OUT', 'TRANSFER']

Min amount transacted when Fraud is set= 0.0

Max amount transacted in a TRANSFER where Fraud is not set= 92445516.64


In [14]:
print('\nThe number of TRANSFERs where Fraud = 0, yet oldbalanceDest = 0 and\
 newBalanceDest = 0: {}'.\
format(len(dfTransfer.loc[(dfTransfer.Fraud == 0) & \
(dfTransfer.oldbalanceDest == 0) & (dfTransfer.newbalanceDest == 0)]))) # 4158


The number of TRANSFERs where Fraud = 0, yet oldbalanceDest = 0 and newBalanceDest = 0: 65


In [15]:
print('\nMin, Max of oldBalanceOrig for Fraud = 1 TRANSFERs: {}'.\
format([round(dfFlagged.oldbalanceOrg.min()), round(dfFlagged.oldbalanceOrg.max())]))

print('\nMin, Max of oldBalanceOrg for Fraud = 0 TRANSFERs where \
oldbalanceOrg = \
newbalanceOrig: {}'.format(\
[dfTransfer.loc[(dfTransfer.Fraud == 0) & (dfTransfer.oldbalanceOrg \
== dfTransfer.newbalanceOrig)].oldbalanceOrg.min(), \
round(dfTransfer.loc[(dfTransfer.Fraud == 0) & (dfTransfer.oldbalanceOrg \
               == dfTransfer.newbalanceOrig)].oldbalanceOrg.max())]))


Min, Max of oldBalanceOrig for Fraud = 1 TRANSFERs: [0, 59585040]

Min, Max of oldBalanceOrg for Fraud = 0 TRANSFERs where oldbalanceOrg = newbalanceOrig: [0.0, 0]


In [16]:
print('\nHave originators of transactions flagged as fraud transacted more than \
once? {}'\
.format((dfFlagged.nameOrig.isin(pd.concat([dfNotFlagged.nameOrig, \
                                    dfNotFlagged.nameDest]))).any())) # False

print('\nHave destinations for transactions flagged as fraud initiated\
 other transactions? \
{}'.format((dfFlagged.nameDest.isin(dfNotFlagged.nameOrig)).any())) # False

# Since only 2 destination accounts of 16 that have 'isFlaggedFraud' set have been
# destination accounts more than once,
# clearly 'isFlaggedFraud' being set is independent of whether a 
# destination account has been used before or not

print('\nHow many destination accounts of transactions flagged as fraud have been \
destination accounts more than once?: {}'\
.format(sum(dfFlagged.nameDest.isin(dfNotFlagged.nameDest)))) # 2


Have originators of transactions flagged as fraud transacted more than once? True

Have destinations for transactions flagged as fraud initiated other transactions? True

How many destination accounts of transactions flagged as fraud have been destination accounts more than once?: 3007


In [17]:
print('\nAre there any merchants among originator accounts for CASH_IN \
transactions? {}'.format(\
(train.loc[train.type == 'CASH_IN'].nameOrig.str.contains('M')).any())) # False


Are there any merchants among originator accounts for CASH_IN transactions? False


In [18]:
print('\nAre there any merchants among destination accounts for CASH_OUT \
transactions? {}'.format(\
(train.loc[train.type == 'CASH_OUT'].nameDest.str.contains('M')).any())) # False


Are there any merchants among destination accounts for CASH_OUT transactions? False


In [19]:
print('\nAre there merchants among any originator accounts? {}'.format(\
      train.nameOrig.str.contains('M').any())) # False

print('\nAre there any transactions having merchants among destination accounts\
 other than the PAYMENT type? {}'.format(\
(train.loc[train.nameDest.str.contains('M')].type != 'PAYMENT').any())) # False


Are there merchants among any originator accounts? False

Are there any transactions having merchants among destination accounts other than the PAYMENT type? False


In [20]:
print('\nWithin fraudulent transactions, are there destinations for TRANSFERS \
that are also originators for CASH_OUTs? {}'.format(\
(dfFraudTransfer.nameDest.isin(dfFraudCashout.nameOrig)).any())) # False
dfNotFraud = train.loc[train.Fraud == 0]


Within fraudulent transactions, are there destinations for TRANSFERS that are also originators for CASH_OUTs? False


In [58]:
print('\nFraudulent TRANSFERs whose destination accounts are originators of \
genuine CASH_OUTs: \n\n{}'.format(dfFraudTransfer.loc[dfFraudTransfer.nameDest.\
isin(dfNotFraud.loc[dfNotFraud.type == 'CASH_OUT'].nameOrig.drop_duplicates())]))


Fraudulent TRANSFERs whose destination accounts are originators of genuine CASH_OUTs: 

              id step      type  amount     nameOrig oldbalanceOrg  \
1139486  1139485  738  TRANSFER  814690  C2029041842        814690   

        newbalanceOrig     nameDest oldbalanceDest newbalanceDest Fraud  
1139486              0  C1023330867              0              0     1  


In [21]:
print('\nFraudulent TRANSFER to C423543548 occured at step = 486 whereas \
genuine CASH_OUT from this account occured earlier at step = {}'.format(\
dfNotFraud.loc[(dfNotFraud.type == 'CASH_OUT') & (dfNotFraud.nameOrig == \
                           'C1023330867')].step.values)) # 185


Fraudulent TRANSFER to C423543548 occured at step = 486 whereas genuine CASH_OUT from this account occured earlier at step = [132]


In [55]:
X = train.loc[(train.type == 'TRANSFER') | (train.type == 'CASH_OUT')]

In [56]:
# X = train.loc[(train.type == 'TRANSFER') | (train.type == 'CASH_OUT')]

randomState = 5
np.random.seed(randomState)

#X = X.loc[np.random.choice(X.index, 100000, replace = False)]

Y = X['Fraud']
del X['Fraud']

In [57]:
# Eliminate columns shown to be irrelevant for analysis in the EDA
X = X.drop(['nameOrig', 'nameDest'], axis = 1)

# Binary-encoding of labelled data in 'type'
X.loc[X.type == 'TRANSFER', 'type'] = 0
X.loc[X.type == 'CASH_OUT', 'type'] = 1
X.type = X.type.astype(int) # convert dtype('O') to dtype(int)

[^1]: [Predicting Fraud in Financial Payment Services | Kaggle](https://www.kaggle.com/arjunjoshua/predicting-fraud-in-financial-payment-services)

In [58]:
X.head()

,id,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
3,2,228,1,39688.11,0.0,0.0,168913.26,208601.37
4,3,229,1,179016.96,103485.0,0.0,2483034.47,2662051.42
5,4,709,1,77155.99,143307.0,66151.01,21213865.83,21291021.82
6,5,205,1,542294.41,0.0,0.0,3875179.23,4417473.64
9,8,154,1,3106.77,0.0,0.0,1002812.92,914313.38


In [59]:
Y.head()

3    0.0
4    0.0
5    0.0
6    0.0
9    0.0
Name: Fraud, dtype: object

In [60]:
Xfraud = X.loc[Y == 1]
XnonFraud = X.loc[Y == 0]
print('\nThe fraction of fraudulent transactions with \'oldbalanceDest\' = \
\'newbalanceDest\' = 0 although the transacted \'amount\' is non-zero is: {}'.\
format(len(Xfraud.loc[(Xfraud.oldbalanceDest == 0) & \
(Xfraud.newbalanceDest == 0) & (Xfraud.amount)]) / (1.0 * len(Xfraud)))) # 0.4955558261293072

print('\nThe fraction of genuine transactions with \'oldbalanceDest\' = \
newbalanceDest\' = 0 although the transacted \'amount\' is non-zero is: {}'.\
format(len(XnonFraud.loc[(XnonFraud.oldbalanceDest == 0) & \
(XnonFraud.newbalanceDest == 0) & (XnonFraud.amount)]) / (1.0 * len(XnonFraud)))) # 0.0006176245277308345


The fraction of fraudulent transactions with 'oldbalanceDest' = 'newbalanceDest' = 0 although the transacted 'amount' is non-zero is: 0.4961832061068702

The fraction of genuine transactions with 'oldbalanceDest' = newbalanceDest' = 0 although the transacted 'amount' is non-zero is: 0.0005850825439118876


In [61]:
X.loc[(X.oldbalanceDest == 0) & (X.newbalanceDest == 0) & (X.amount != 0), \
      ['oldbalanceDest', 'newbalanceDest']] = - 1

In [62]:
X.loc[(X.oldbalanceOrg == 0) & (X.newbalanceOrig == 0) & (X.amount != 0), \
      ['oldbalanceOrg', 'newbalanceOrig']] = np.nan

In [65]:
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import average_precision_score
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance, to_graphviz

In [69]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, \
                                                random_state = randomState)

In [70]:
shape(x_train),shape(y_train),shape(x_test),shape(y_test)

NameError: name 'shape' is not defined

In [67]:
# Long computation in this cell (~1.8 minutes)
weights = (Y == 0).sum() / (1.0 * (Y == 1).sum())
clf = XGBClassifier(max_depth = 3, scale_pos_weight = weights, \
                n_jobs = 4)
probabilities = clf.fit(trainX, trainY).predict_proba(testX)
print('AUPRC = {}'.format(average_precision_score(testY, \
                                              probabilities[:, 1])))

TypeError: '>' not supported between instances of 'float' and 'str'

In [ ]:
submission = pd.DataFrame()
submission['id'] = test_id
submission['Fraud'] = 您的預測結果
submission.to_csv('您的檔名.csv',index=False)